# Ikeda barge

# Purpose
Ikeda has proposed a formula to calcualte the eddy damping for barges: 
$$ B_{E}=\left(\frac{2}{\pi}\right) \rho L d^{4}\left(H_{0}^{2}+1-\frac{O G}{d}\right)\left(H_{0}^{2}+\left(1-\frac{O G}{d}\right)^{2}\right) R_{0} \omega $$


# Methodology
Investigate if this formulat makes any sense to the KVLCC2

# Setup

In [ ]:
# %load imports.py
"""
These is the standard setup for the notebooks.
"""

%matplotlib inline
%load_ext autoreload
%autoreload 2

from jupyterthemes import jtplot
jtplot.style(theme='onedork', context='notebook', ticks=True, grid=False)

import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
import os
import matplotlib.pyplot as plt
from collections import OrderedDict
#plt.style.use('paper')

#import data
import copy
from mdldb.run import Run

from sklearn.pipeline import Pipeline
from rolldecayestimators.transformers import CutTransformer, LowpassFilterDerivatorTransformer, ScaleFactorTransformer, OffsetTransformer
from rolldecayestimators.direct_estimator_cubic import EstimatorQuadraticB, EstimatorCubic
from rolldecayestimators.ikeda_estimator import IkedaQuadraticEstimator
import rolldecayestimators.equations as equations
import rolldecayestimators.lambdas as lambdas
from rolldecayestimators.substitute_dynamic_symbols import lambdify
import rolldecayestimators.symbols as symbols
import sympy as sp

from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex

from sklearn.metrics import r2_score
from src.data import database
from mdldb import tables
import shipflowmotionshelpers.shipflowmotionshelpers as helpers


In [ ]:
from rolldecayestimators import ikeda_barge

In [ ]:
db = database.get_db()
run = db.session.query(Run).get(21338)

In [ ]:
scale_factor = run.model.scale_factor
w = 2.4621496310208086/np.sqrt(scale_factor)
draught = (run.loading_condition.TA + run.loading_condition.TF)/2
OG = run.loading_condition.kg - draught
phi_as = np.deg2rad(np.linspace(0,10,100))
beam = run.ship.beam
rho = 1000
g = 9.81
parameters = {
    'rho' : rho,
    'lpp' : run.ship.lpp,
    'd' : draught,
    'beam' : beam,
    'OG' : OG,
    'phi_a' : phi_as, 
    'w' : w,
}

df = pd.DataFrame(index=phi_as)
df['B_E'] = ikeda_barge.eddy(**parameters)
Disp = run.loading_condition.Volume
df['B_E_hat'] = lambdas.B_to_hat_lambda(B=df['B_E'], Disp=Disp, beam=beam, g=g, rho=rho)


In [ ]:
fig,ax=plt.subplots()
df.plot(y='B_E_hat', ax=ax)